# In this file we assume the rules were given by an expert.

This is as close as the PFS system gets to the workflow of the `normal` Fuzzy System Workflow. Please check out notebook 1 and 2 if you haven't already. It will make more sense if you read the notebooks in recommended order. What this file illustrated is how you could make prediction in a non-automatic way using rules defined by an expert for example.

### Imports

In [1]:
import pickle
import numpy as np
from matplotlib import pyplot as plt
from math import isclose
from numpy.lib.function_base import append
from simpful import *
from simpful.rules import swap_none
from modules.evolutiongp_multi__ import FuzzyEvolution
from numpy import load
import time
import regex as re

### Load rules

In [2]:
with open('rules.pkl', 'rb') as input:
    rules = pickle.load(input)

#### Simulate expert knowledge by taking the best model we found in the automatic rule search

#### (act as if the automatic generated model was given by an expert)

In [3]:
rules

['IF (fare IS cluster0) OR (passclass IS cluster0) AND_p (parentchild IS cluster0) THEN P(OUTCOME IS DEATH)=0.617, P(OUTCOME IS ALIVE)=0.383',
 'IF (NOT (sex IS cluster1)) AND (embarked IS cluster1) OR (passclass IS cluster1) AND (age IS cluster1) THEN P(OUTCOME IS DEATH)=0.0, P(OUTCOME IS ALIVE)=1.0',
 'IF (NOT (parentchild IS cluster2)) THEN P(OUTCOME IS DEATH)=0.71, P(OUTCOME IS ALIVE)=0.29']

### Modelling with Simpful

#### Data loads

In [4]:
# load X_train observation_based
X = np.load('./inputs_processed_/X_train.npy')

# X_test observation bassed
X_T = np.load('./inputs_processed_/X_test.npy')

# correspinding y_train and test
y = np.load("./inputs_processed_/y_train.npy").tolist()
y = [item for sublist in y for item in sublist]
y_test = np.load("./inputs_processed_/y_test.npy").tolist()
y_test = [item for sublist in y_test for item in sublist]

### Initialisation

#### It is important to maintain the order of the dataset when specifying the vars. The order of the consequents should correspond to the way the rules were defined. E.g. :

1. 'IF (fare IS cluster0) OR (passclass IS cluster0) AND_p (parentchild IS cluster0) THEN P(OUTCOME IS DEATH)=0.617, P(OUTCOME IS ALIVE)=0.383'

Death should therefore be the first consequent term in each rule and alive should be the second!

In [7]:
# Initialize using var_names/ consequents
vars = ['passclass', 'sex', 'age', 'sibling', 'parentchild', 'fare', 'embarked']
consequents = ['DEATH', 'ALIVE']

#### Assuming we have a test set and a train set we can initialise as follows. Many options are possible. For more information we recommend reading the docstring.

In [9]:
expert_model = ProbaFuzzySystem(consequents=consequents, var_names=vars, X=X, X_test=X_T, y=y, y_test=y_test, pred_test=True, _return_class=True, generateprobas=False)

In [18]:
# #Comment out for docstring on ProbaFuzzySystem

# help(ProbaFuzzySystem)

#### Here we load the rules that we generated using automatic modelling. The idea is however that you can specify whatever rules you like with whatever probabilities you like. To show however that it leads to the same results as a verification step here we use the model which was found in notebook 1: 1) Titanic Model - Automatic modeling using Genetic Programming

- As can be seen below the accuracy matches, for the comparison check out file 2) which shows an accuracy of 0.8100558659217877 and compare it for yourself to the one shown below (in the last cell).

In [10]:
expert_model.add_proba_rules(rules)

#### What X_reformatter is make sure the rules are interpreted in the right way (but in the order of the `` vars `` variable. You usually give the whole dataset and the variables will be read correctly regardless thanks to the X_reformatter function.

In [11]:
expert_model.X_reformatter()

#### This is a convenience method for adding the right linguistic variables all at once  (Keep in mind that the datasets, variables and all relevant information was loaden during intialisation.)

In [12]:
expert_model.add_linguistic_variables()

#### Make predictions (Keep in mind that the datasets, variables and all relevant information was loaden during intialisation.)

In [13]:
preds = expert_model.predict_pfs()

#### The predictions below are based on the test set (as user defined: ``pred_test=True")

In [14]:
preds

[0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0]

In [15]:
expert_model.evaluate_accuracy()

0.8100558659217877